In [1]:
%reload_ext autoreload
%autoreload 2

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "3"

import pandas as pd
from datasets import load_from_disk
from transformers import TrainingArguments
from src.model.trainer_protein_subset import ProteinSampleSubsetTrainer
import torch
from torch import nn

In [2]:
if os.path.exists("../tmp/data/train_val_GO_small_subset"):
    dataset_path = "../tmp/data/train_val_GO_small_subset"
    dataset = load_from_disk(dataset_path)
else:
    dataset_path = "../tmp/data/train_val_GO"
    dataset = load_from_disk(dataset_path)
    dataset = dataset["train"]
    dataset = dataset.select(range(2000))
    dataset.save_to_disk("../tmp/data/train_val_GO_small_subset")

In [12]:
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.dummy_param = nn.Parameter(torch.zeros(1))

    def forward(self, *args, **kwargs):
        # print(args)
        # print(kwargs)
        return {"loss": torch.tensor(0.0)} 

model = SimpleModel()

def data_collator(features):
    return features

In [ ]:
dataset[0]

In [ ]:
training_args = TrainingArguments(
    output_dir="./test_output",
    per_device_train_batch_size=2,
    auto_find_batch_size=True,
    num_train_epochs=5,
    logging_dir="./logs",
    logging_steps=1,
    remove_unused_columns=False,
)

trainer = ProteinSampleSubsetTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=None,  # Not testing evaluation here
    tokenizer=None,
    data_collator=data_collator,
)

In [ ]:
trainer.train()

In [7]:
# len(next(iter(trainer.get_train_dataloader())))

In [8]:
ttt = [x for x in trainer.get_train_dataloader()]

In [ ]:
sum([len(x) for x in ttt])

In [ ]:
print(len(ttt))
print()
for x in ttt:
    print(len(x))

In [ ]:
[x['identifier'] for x in ttt[0]] + [x['identifier'] for x in ttt[1]]

In [ ]:
# len(set([x['identifier'] for batch in ttt for x in batch]))

In [ ]:
output_data = []
for epoch in range(int(training_args.num_train_epochs)):
    dataloader = trainer.get_train_dataloader()
    for batch in dataloader:
        for example in batch:
            output_data.append({
                "epoch": epoch,
                "identifier": example["identifier"],
                "GO term": example["term"]
            })
        break


In [ ]:
df = pd.DataFrame(output_data)
display(df)

In [ ]:
df['identifier'].value_counts()